# Learning $\hat{\Omega}$

In this notebook, we're going to demonstrate parameter inference for $\Omega$, under the assumption that $\Omega$ depends only on subgroup sizes within each edge. We consider a scenario in which we already know or have a guess about $Z$. 

In [44]:
using Pkg; Pkg.activate(".")
using HypergraphModularity

using NLopt
using StatsBase
using Optim

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


First thing we'll do is define a set of parameters, including group memberships $Z$, degree parameters $\vartheta$, and the connection function $\Omega$. 

In [109]:
n = 100
Z = rand(1:5, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)
μ = mean(ϑ)
kmax = 4;

function ω(p, α)
    k = sum(p)
    return sum(p)/sum((p .* (1:length(p)).^α[k])) / n^(α[kmax+k]*k)
end

function ∇ω(p, α)
    
    g = zero(α)
    
    k = sum(p)
    j = length(p)
    
    c = sum(p.*(1:j).^α[k])
    d = sum(p.*(1:j).^α[k].*log.(1:j))
    
    g[k] = d/c*ω(p, α)
    g[k+kmax] = log(n)*k*ω(p,α)
    
    return -convert(Vector{Float64}, g)
end

α = vcat(repeat([5.0], kmax), 0.2*(1:kmax))
Ω = partitionIntensityFunction(ω, kmax; grad = ∇ω);
# Ω = partitionIntensityFunction(ω, kmax; grad = nothing);

H = sampleSBM(Z, ϑ, Ω;α=α, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((100,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
  E: Dict{Int64,Dict}
  D: Array{Int64}((100,)) [4, 8, 9, 6, 3, 7, 2, 6, 6, 4  …  8, 7, 2, 6, 10, 4, 8, 9, 3, 5]


In [110]:
function formObjective_(H, Z, Ω)
    ℓ       = maximum(Z)
    C       = evalCuts(H,Z,Ω)
    V, μ, S = evalSums(Z,H,ℓ,true);
    Ŝ       = HypergraphModularity.aggregateSums(S,Ω)
    
    function f(α)
        obj = 0.0
        for p in keys(Ŝ)
            Op   = Ω.ω(p, α)
            obj += get(C, p, 0)*log(Op) - Ŝ[p]*Op
        end
        obj = -convert(Float64, obj)
#         println(obj, α)
        return obj # sign is for minimization
    end

    if Ω.grad != nothing     
        function g!(grad, α)
            obj = 0.0
            grad[:] = zero(α)
            for p in keys(Ŝ)
                Op    = Ω.ω(p, α)
                obj  += get(C, p, 0)*log(Op) - Ŝ[p]*Op
                grad = grad + (get(C, p, 0)/Op - Ŝ[p]).*Ω.grad(p, α)
            end
            grad = -convert(Vector{Float64}, grad)
#             println(grad)
            return grad # sign is for minimization
        end
        return f, g!
    else
        return f
    end
end

formObjective_ (generic function with 1 method)

In [117]:
function learnParameters_(H, Z, Ω, α0; verbose = false, ftol_abs = 1e-6, maxeval = 100000)
    """
    a more reliable optimization to learn parameters given a partition, using the COBYLA algorithm from NLopt. 
    returns both the optimal α and the objective value at that point. 
    """
    
    kmax = length(α0) ÷ 2
    f, g!  = formObjective_(H, Z, Ω)
    
    res = Optim.optimize(f, α0, BFGS())
    return(res)
end

learnParameters_ (generic function with 1 method)

In [120]:
res = learnParameters_(H, Z, Ω, α0)

 * Status: success

 * Candidate solution
    Final objective value:     2.550967e+03

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 6.98e-10 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.24e-10 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 3.75e-08 ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    22
    f(x) calls:    99
    ∇f(x) calls:   99
